<a href="https://colab.research.google.com/github/jbtuku/Data-Engineering/blob/main/Datatransformation/enhance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from google.colab import files
from datetime import datetime, timedelta


uploaded = files.upload()
filename = list(uploaded.keys())[0]
df = pd.read_csv(filename)


df['OPD_DATE'] = pd.to_datetime(df['OPD_DATE'].str[:9], format='%d%b%Y')
df['TIMESTAMP'] = df['OPD_DATE'] + pd.to_timedelta(df['ACT_TIME'], unit='s')
df.drop(columns=['OPD_DATE', 'ACT_TIME'], inplace=True)


df['dMETERS'] = df['METERS'].diff()
df['dTIMESTAMP'] = df['TIMESTAMP'].diff().dt.total_seconds()

df['SPEED'] = df.apply(
    lambda row: row['dMETERS'] / row['dTIMESTAMP'] if row['dTIMESTAMP'] and row['dTIMESTAMP'] > 0 else 0,
    axis=1
)


df.drop(columns=['dMETERS', 'dTIMESTAMP'], inplace=True)


print(" SPEED Summary:")
print(f"Minimum Speed: {df['SPEED'].min():.2f} m/s")
print(f"Maximum Speed: {df['SPEED'].max():.2f} m/s")
print(f"Average Speed: {df['SPEED'].mean():.2f} m/s")


print("\nSample rows with SPEED:")
print(df[['TIMESTAMP', 'METERS', 'SPEED']].head())


Saving bc_trip259172515_230215.csv to bc_trip259172515_230215.csv
 SPEED Summary:
Minimum Speed: 0.00 m/s
Maximum Speed: 17.40 m/s
Average Speed: 7.18 m/s

Sample rows with SPEED:
            TIMESTAMP  METERS  SPEED
0 2023-02-15 05:41:09      40    0.0
1 2023-02-15 05:41:14      48    1.6
2 2023-02-15 05:41:19      57    1.8
3 2023-02-15 05:41:24      73    3.2
4 2023-02-15 05:41:29     112    7.8


In [2]:
import pandas as pd
from google.colab import files
from datetime import timedelta


uploaded = files.upload()
filename = list(uploaded.keys())[0]


df = pd.read_csv(filename)


df['OPD_DATE'] = pd.to_datetime(df['OPD_DATE'].str[:9], format='%d%b%Y')
df['TIMESTAMP'] = df['OPD_DATE'] + pd.to_timedelta(df['ACT_TIME'], unit='s')


df = df.sort_values(by=['EVENT_NO_TRIP', 'TIMESTAMP'])


df['dMETERS'] = df.groupby('EVENT_NO_TRIP')['METERS'].diff()
df['dTIMESTAMP'] = df.groupby('EVENT_NO_TRIP')['TIMESTAMP'].diff().dt.total_seconds()

df['SPEED'] = df.apply(
    lambda row: row['dMETERS'] / row['dTIMESTAMP'] if row['dTIMESTAMP'] and row['dTIMESTAMP'] > 0 else 0,
    axis=1
)

df.drop(columns=['dMETERS', 'dTIMESTAMP'], inplace=True)


max_speed = df['SPEED'].max()
max_row = df[df['SPEED'] == max_speed].iloc[0]
max_time = max_row['TIMESTAMP']
max_location = (max_row['GPS_LATITUDE'], max_row['GPS_LONGITUDE'])


median_speed = df['SPEED'].median()


print(" Vehicle #4223 on Feb 15, 2023:")
print(f"Maximum Speed: {max_speed:.2f} m/s")
print(f"Occurred at: {max_time}")
print(f"Location: Latitude {max_location[0]}, Longitude {max_location[1]}")
print(f"Median Speed: {median_speed:.2f} m/s")


Saving bc_veh4223_230215.csv to bc_veh4223_230215.csv
 Vehicle #4223 on Feb 15, 2023:
Maximum Speed: 17.40 m/s
Occurred at: 2023-02-15 05:44:49
Location: Latitude 45.505452, Longitude -122.660822
Median Speed: 7.20 m/s
